In [28]:
import numpy as np
import pandas as pd
#import random
import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
#import jieba
#import jieba.analyse
#import gensim
#from sklearn.manifold import TSNE

from sklearn.cross_validation import train_test_split

%matplotlib notebook

plt.rcParams['font.sans-serif']=['SimHei'] # chinese

In [134]:
import apriori
import fpgrowth as fpg
from fastFM import als
from scipy import sparse

In [186]:
import imp
fpg = imp.reload(fpg)
apriori = imp.reload(apriori)

In [93]:
def build_fp_tree(file, support=2):
    data = fpg.loadData(file)
    tree, header = fpg.createTree(data, minSup=support)
    return tree, header

In [31]:
def run_apriori(file, support=0.1, confidence=0.6):
    data_gen = apriori.dataFromFile(file)
    items, rules = apriori.runApriori(data_gen, support, confidence)
    #apriori.printResults(items, rules)
    return items, rules

In [192]:
large_itemsets, recomm_rules = run_apriori('../data/sample_simple.csv', 0.2, 0.8)
apriori.printResults(large_itemsets, recomm_rules, fn_items='out_large_itemsets.csv', fn_rules='out_recomm_rules.csv')

item: ('mango',) , 0.250
item: ('chicken',) , 0.250
item: ('rice', 'apple') , 0.250
item: ('beer', 'chicken') , 0.250
item: ('milk', 'rice') , 0.250
item: ('rice', 'chicken') , 0.250
item: ('milk', 'rice', 'beer') , 0.250
item: ('beer', 'chicken', 'rice') , 0.250
item: ('rice', 'apple', 'beer') , 0.250
item: ('milk', 'beer') , 0.375
item: ('beer', 'apple') , 0.375
item: ('milk',) , 0.500
item: ('apple',) , 0.500
item: ('rice',) , 0.500
item: ('beer', 'rice') , 0.500
item: ('beer',) , 0.750

------------------------ RULES:
Rule: ('chicken',) ==> ('beer',) , 1.000
Rule: ('rice',) ==> ('beer',) , 1.000
Rule: ('chicken',) ==> ('rice',) , 1.000
Rule: ('milk', 'rice') ==> ('beer',) , 1.000
Rule: ('chicken',) ==> ('beer', 'rice') , 1.000
Rule: ('beer', 'chicken') ==> ('rice',) , 1.000
Rule: ('rice', 'chicken') ==> ('beer',) , 1.000
Rule: ('rice', 'apple') ==> ('beer',) , 1.000


In [140]:
#run_apriori('../data/sample_integrated.csv', 0.2, 0.8)

In [164]:
tree, header = build_fp_tree('../data/sample_simple.csv')

In [165]:
tree.disp(fname='out_fptree.txt')

-Null Set:1
--beer:6
---apple:3
----rice:2
-----chicken:1
---milk:3
----rice:2
-----chicken:1
--apple:1
---mango:1
--milk:1
---mango:1


In [36]:
df = pd.read_csv('../data/sample_simple.csv', header=None)
df

,0,1,2,3
0,apple,beer,rice,chicken
1,apple,beer,rice,NaN
2,apple,beer,NaN,NaN
3,apple,mango,NaN,NaN
4,milk,beer,rice,chicken
5,milk,beer,rice,NaN
6,milk,beer,NaN,NaN
7,milk,mango,NaN,NaN


In [37]:
def create_item_set(df):
    itemset = set()
    for index, row in df.iterrows():
        for e in row.tolist():
            itemset.add(e)
    itemset.remove(np.nan)
    return itemset

In [11]:
itemset = create_item_set(df)
itemset

{'apple', 'beer', 'chicken', 'mango', 'milk', 'rice'}

In [12]:
def create_onehot_vec(itemset):
    items = list(itemset)
    itemvec = sparse.eye(len(items))
    itemvec_dict = {}
    for i in range(len(items)):
        itemvec_dict[items[i]] = itemvec.getrow(i)
    return itemvec_dict

In [13]:
itemvec = create_onehot_vec(itemset)
itemvec

{'apple': <1x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'beer': <1x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'chicken': <1x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'mango': <1x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'milk': <1x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'rice': <1x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>}

In [14]:
(itemvec["beer"] + itemvec["mango"]).todense()

matrix([[0., 0., 0., 0., 1., 1.]])

In [15]:
uservec = create_onehot_vec(df.index)

In [16]:
uservec

{0: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 1: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 2: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 3: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 4: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 5: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 6: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 7: <1x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>}

In [17]:
def create_user_item_matrix(df, uservec, itemvec, mimic="SVD++"):
    mat = None
    for index, row in df.iterrows():
        vu = uservec[index]
        vi_sum = None
        if mimic != "MF":
            for item_name in row.tolist():
                if item_name is np.nan:
                    continue
                vi = itemvec[item_name]
                if vi_sum is None:
                    vi_sum = vi
                else:
                    vi_sum += vi
        if mimic == "SVD++":
            vi_sum /= np.sqrt(np.sum(vi_sum))
        for item_name in row.tolist():
            if item_name is np.nan:
                continue
            vi = itemvec[item_name]
            if mimic == "SVD++":
                vuil = sparse.hstack((vu, vi, vi_sum))
            elif mimic == "MF":
                vuil = sparse.hstack((vu, vi))
            if mat is None:
                mat = vuil
            else:
                mat = sparse.vstack((mat, vuil))
    return mat

In [39]:
mat = create_user_item_matrix(df, uservec, itemvec, mimic="SVD++")
#mat = create_user_item_matrix(df, uservec, itemvec, mimic="MF")
#mat.todense()

In [40]:
X = mat
y = np.ones(mat.shape[0])
X_ = sparse.csc_matrix(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y)

In [41]:
fm = als.FMRegression(n_iter=200, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [42]:
y_pred

array([0.99990039, 1.00009286, 0.99978987, 0.99993868, 0.99998846,
       1.00043402])

In [22]:
###

In [49]:
df_puffer = pd.read_excel('180419mark_satisfied_puffer.xlsx')

In [50]:
dfp = df_puffer.copy()

In [55]:
df1 = dfp[dfp['mark'] == 1].copy()

In [56]:
def mark_sid_type(row):
    botid = row['source_type']
    sid = row['sid']
    if botid == "audio_music":
        row['sid'] = "m" + sid
    elif botid == "audio_unicast":
        row['sid'] = "u" + sid
    elif botid == "ai.dueros.bot.short_video":
        row['sid'] = "s" + sid
    elif botid == "ai.dueros.bot.video_on_demand":
        row['sid'] = "v" + sid
    return row

dfs = df1.apply(mark_sid_type, axis=1)

In [74]:
df_sidlookup = dfs[['sid', 'slots', 'content', 'nlu']].set_index('sid')
df_sidlookup.to_excel('out_sid_lookup.xlsx')
df_sidlookup.head()

In [153]:
df1 = df_sidlookup.reset_index()

In [156]:
len(df1['sid'])

42180

In [85]:
df_sid = dfs[['uid', 'sid']].groupby('uid').agg(lambda x: ','.join(x))

In [86]:
df_sid.to_csv('out_uid_sid.csv', index=False, header=None, sep='|')

In [183]:
tree, header = build_fp_tree('out_uid_sid.csv', support=10)

In [184]:
tree.disp(fname='out_fptree.txt', df=df_sidlookup)

-Null Set:1
--m991053430:47
---m1329025041:1
----m993189565:1
-----m1019301623:1
------v37d8667654cc4645b451236551db1141:1
---m1365519490:1
----m1233414257:1
-----m984599676:1
------m1299328953:1
---m1025707874:1
----m1029883736:1
-----m1306305360:1
------m1046389703:1
-------m1043344208:1
--------m1322433210:1
---------m1248969980:1
---m1019301623:1
----m1013613347:1
---m1281941647:1
----s11302681162812298723:1
-----m1322431686:1
------m1117405477:1
---m1001219330:1
----m1241391273:1
-----s11302681162812298723:1
------m1360526302:1
-------m1051992469:1
--------m1004106362:1
---m1022000981:3
----m1055547303:1
-----m1233419417:1
------m1370494604:1
----m1012283935:1
-----m1360527319:1
------u64061868578:1
-------u57803345634:1
--------m1003309624:1
---------m1010209304:1
----m1224063886:1
-----m1005864835:1
------v6504abd4b4714848877c9dcc8b283c7d:1
---m964989942:1
----m1092828664:1
---m1014804743:1
----m1023707444:1
-----m1284968694:1
---m1004725438:1
----m1378473143:1
-----m1370492555:

------s1947196397967195075:1
-------m1136066881:1
--------m990590106:1
---------m1052156413:1
----------s794585456984438506:1
-----------s11725220996556825632:1
------------m1345975879:1
-------------m1233414257:1
--------------m1120469363:1
---------------m1322432055:1
----------------m1009061423:1
-----------------m1036645627:1
------------------m1360501102:1
-------------------m968849604:1
--------------------m1360526008:1
---------------------m1375860059:1
---m1329836160:3
----m1001219330:1
-----m1016280242:1
------s11409760776512355215:1
-------m1026716815:1
--------s11673923141555725688:1
---------m1016039543:1
----------va7b28e07aa644a45abfc35a99895d8ab:1
-----------u57777338375:1
------------m1019879977:1
-------------m1101099312:1
--------------u57474520359:1
---------------m1217331891:1
----------------v7527503d343440f89bc4ff7a79dfc9fa:1
----m1141773023:1
----m1322431587:1
-----m1001165755:1
------m1019177746:1
-------m977281875:1
--------m1133144120:1
---------s3822058106164

-----m994482868:1
------m998663430:1
-------u64061868578:1
--------m1340260609:1
---------m1004225499:1
----------m1004975420:1
----m1029883736:1
-----m1322431554:1
------m988942596:1
----m1375860059:1
-----m1322433213:1
---m1365519490:4
----m993189565:1
-----m1360526302:1
------m1360527307:1
-------m1051992469:1
--------m973308537:1
---------s1525724261775906673:1
----------v736fc53c858e41a6b29150c6023535df:1
----m1001219330:2
-----m992308257:1
------m1360526302:1
-------m1005435024:1
--------m1009362125:1
---------sweishenmehuixiayu:1
----------m1056927930:1
-----------m1346751859:1
-----m1322431689:1
------m1345521604:1
-------m1004226551:1
--------m1318134941:1
---------m1087883560:1
----------m1108384828:1
----m998144258:1
-----m987285300:1
------m1053702444:1
-------m1291627343:1
--------m1001066635:1
---------m1321376200:1
----------m1322432199:1
-----------m1327803153:1
---m1356820310:1
----m990759337:1
-----m1025708042:1
------s11673923141555725688:1
-------s151785725293246291

----------------m1027438381:1
-----------------m984599676:1
------------------m1322433222:1
-------------------m1306138053:1
---m1343057863:1
----m1092128721:1
-----s7678527194255808147:1
---m1322431794:3
----m1032230936:1
-----m1356055436:1
------m1015749748:1
-------m1043344208:1
--------m1004225499:1
---------m967749900:1
----m1356820310:1
-----m1289716254:1
------m996417485:1
----m1327881243:1
-----m1322431584:1
---u58356824385:1
----m1322432142:1
-----m1322433483:1
------m1022054770:1
---m1039015538:1
----s912338179177576920:1
-----u64061868578:1
---m1092128721:1
----m1005864835:1
-----s12765656016992815457:1
---m1124483944:1
---s1947196397967195075:1
----u56486568164:1
-----u57779605418:1
------u85040552425:1
---m982501848:1
----m997120535:1
-----m1356820310:1
------m1016280242:1
-------vb2d6200694e1c59e15b449c60c9626e2:1
--------m1322431587:1
---------m1092128721:1
----------m1351721093:1
-----------m980091060:1
------------m990827104:1
-------------v74d1ccf8d9f148a68b75507e6a2b

------m1020037885:1
-------v18b17fb2840f48818d41d979da3ecb54:1
--------m968206422:1
---------u85040552425:1
----------va5511a917451e0ee5cafaed0d7d1adcd:1
---u58309976738:1
----m997374012:1
-----m1004226551:1
------s15760024689030870704:1
-------m977208744:1
---m1365519490:2
----v679cb853037f453cb83a4599e3072dd2:1
-----m1370393099:1
------m1356820310:1
-------m1290719507:1
--------v85a2e3e44bb3a034c9c171ac656a8a3b:1
---------m974775711:1
----------m1322431677:1
-----------m1360526764:1
------------m1322433525:1
----m982501848:1
-----m1346157493:1
------m1025904638:1
-------m1301765682:1
--------m1093289710:1
---------m1356775019:1
----------m1345852804:1
-----------m1019301623:1
------------m1097313584:1
-------------m1258386070:1
--------------m1303662297:1
---------------m1215499785:1
----------------m1006343021:1
---m1029095653:1
----m1322431569:1
-----s10977594079269672094:1
------m1016326571:1
-------s1947196397967195075:1
--------m1053702444:1
---------m1002249083:1
----------s105

...

---m994482868:1
----m1291627343:1
-----m1322431584:1
------m964759878:1
-------m1034051867:1
--------m1004615185:1
---vd9d6363fdb284796b160fc55d092a77f:1
---m1004168019:1
---m982501848:1
----m1015994989:1
---m1327881243:1
----m1023350273:1
-----m1125929467:1
------m1322204116:1
-------m1046389703:1
--------vf45505041011499f8b4386a292e77435:1
---------m1006343021:1
----------m1041969545:1
-----------m1336988176:1
---m1356820310:1
----v18b17fb2840f48818d41d979da3ecb54:1
-----m1121915816:1
---m1053702444:1
----m1057213800:1
---m1092108481:1
---m978045018:1
----u64061868578:1
-----s4218627676439904930:1
------u85019006832:1
--m1346157493:7
---m1000802966:1
----m1023707444:1
-----m1013005708:1
------m1037043782:1
-------u66287668262:1
--------m1019879977:1
---------m1322432994:1
----------u57760605834:1
-----------m1012296850:1
---m1368855664:1
----m1351996751:1
-----m1224063886:1
------m1368905050:1
-------m1057213800:1
--------m1313326594:1
---------u61123874783:1
----------m1299452734:1


------------m1322432001:1
-------------m1322433483:1
--------------m1104373169:1
---------------m1290725523:1
----------------m1308118477:1
-----------------m989020308:1
---m1233419417:1
---m1088339200:1
----m1360526188:1
-----m1333239223:1
------m1027205039:1
-------m1041985679:1
--------m1125508961:1
---m1001442387:1
----m1025904638:1
-----m1005075935:1
------m1234925521:1
-------m1014837778:1
--------v37d8667654cc4645b451236551db1141:1
---------m1327484919:1
---m1370492555:1
----m1321376200:1
---s10977594079269672094:1
----m968206422:1
-----m1360526008:1
---u85041092492:1
--m1138580000:2
---m1087883560:1
----m1322432199:1
--m1343057863:13
---s5015996802295282022:1
----m1327802868:1
-----s1080590316673673613:1
------s11261967489787043384:1
-------m1089643462:1
---m1301765682:1
----m1025707874:1
-----m1001165755:1
------m1093289710:1
-------m1322431650:1
--------m1028654668:1
---------m1322433483:1
----------m1322431860:1
-----------m1115427451:1
---m997120535:1
----m974775711:1
-----

-----u56723165451:1
------m1370494604:1
---m1377364526:1
----m990759337:1
-----m1092828664:1
------m1097313584:1
-------m1258386070:1
--------m994483228:1
---------m1217262401:1
---s9524856411733439290:1
----m1322431554:1
-----s13328860339007112925:1
--s5015996802295282022:4
---u85019006832:1
---v7527503d343440f89bc4ff7a79dfc9fa:1
----s7148370175486809820:1
--m1308717867:7
---m1322431587:1
----m1322431554:1
-----m1210630348:1
------u56472117092:1
-------m975558567:1
---m1345852804:1
----m1295833608:1
-----v0d912682afe746d099f249a7ae82b692:1
------m1296231150:1
---v736fc53c858e41a6b29150c6023535df:1
---s11725220996556825632:1
----m1333239223:1
---m1345521604:1
----m1017551038:1
-----m1313326594:1
------m977281875:1
-------u57729293216:1
--m1377364526:8
---m1039015538:1
----m1333239223:1
-----m990410542:1
------m1070505348:1
-------m1121915816:1
---m1301765682:1
----m964798503:1
-----m993179695:1
---m1322432199:1
---m1002249083:1
----m1368905050:1
-----m1360526302:1
------m1092108481:1
-

-----m1008206774:1
---m1346751859:1
----m1087883560:1
-----m1014837778:1
---m1301765682:1
--m1027205039:4
---m1322431554:1
---m1257842790:1
----m1002992845:1
---m1028177873:1
----m1116753052:1
---m978870771:1
--m1051992469:4
---u56486568164:1
----m1018598902:1
---m1322432142:1
----m967727517:1
--m1248564077:1
---m1308118477:1
----m1360526830:1
--m1032230936:3
---m1333239223:1
----m1342110055:1
-----m1023350273:1
------m1377002861:1
-------m1057538385:1
--------m1353484817:1
---------u57751881606:1
---m1003246898:1
----m1016039543:1
-----u56777508521:1
------u57758306961:1
--m1094630094:1
--m1168883514:1
--m1037043782:2
---m1376748200:1
--m1023350273:2
---m1373170442:1
---m1120469363:1
----m1081428732:1
-----m968009601:1
------m1010209304:1
--m1012283935:2
---s6676773808377990636:1
---m1125202424:1
----m1135582310:1
-----u57758306961:1
--m1345521604:1
---m1342110055:1
----m1005075935:1
-----s13674672424966518537:1
--m996292539:1
--m1374848297:2
--m1346865679:1
---m1360527319:1
----m9904

-----m1365771691:1
------m1002992845:1
-------m975558567:1
--u57388718416:1
--m1360527307:1
--m1315481153:1
---m1318134941:1
----m968476020:1
-----m1322433441:1
--m990759337:2
---m1003309624:1
---m977281875:1
----m1321376200:1
-----m996543287:1
------u85040728916:1
--m1144526891:2
--m1133360119:1
--u61124578310:1
--u64061868578:4
---m1101099312:1
----m1075450977:1
---m1019741800:1
----s15760024689030870704:1
-----m1264758453:1
--m1026579049:1
--m1356775019:4
---m1148832347:1
----m964989942:1
-----m1005314929:1
------m1376748221:1
---m1306305360:1
----m1043344208:1
-----m1009061423:1
------m1121915816:1
---m1165922811:1
---m997476695:1
--m1342110055:3
---m1299452734:1
---m1003246898:1
----m1023350273:1
-----m1004226551:1
------m1248564077:1
-------m1290725523:1
---u57751881606:1
----u56777508521:1
--m1375907420:1
---m1017837173:1
--m1290096671:1
--m1039015538:6
---s10670986106136248482:1
---m977281875:1
---m1013472044:1
----m1021574770:1
-----m1017106267:1
------m1017418700:1
-------m12

In [193]:
large_itemsets, recomm_rules = run_apriori('out_uid_sid.csv', 0.001, 0.2)
apriori.printResults(large_itemsets, recomm_rules, fn_items='out_large_itemsets.csv', fn_rules='out_recomm_rules.csv')

item: ('m998806908',) , 0.001
item: ('u57801045216',) , 0.001
item: ('u56488182784',) , 0.001
item: ('m1322432619',) , 0.001
item: ('m1021758128',) , 0.001
item: ('m1322434203',) , 0.001
item: ('m1342991686',) , 0.001
item: ('s1500265151522498120',) , 0.001
item: ('m1308860745',) , 0.001
item: ('vcab14989ef254f82a2dcacfc37b50169',) , 0.001
item: ('m1158666955',) , 0.001
item: ('m1030941281',) , 0.001
item: ('u57740064969',) , 0.001
item: ('m1015931167',) , 0.001
item: ('s3915420001504599418',) , 0.001
item: ('s4587667677272372261',) , 0.001
item: ('m1014558744',) , 0.001
item: ('m1023707342',) , 0.001
item: ('s10088672227633196657',) , 0.001
item: ('m1051470930',) , 0.001
item: ('m1072472904',) , 0.001
item: ('m995081271',) , 0.001
item: ('s8922730674528147087',) , 0.001
item: ('u57739175212',) , 0.001
item: ('vffce05da184144dfae2a2676cb0d6bf2',) , 0.001
item: ('m1370492519',) , 0.001
item: ('s5154927483293264180',) , 0.001
item: ('m1231656243',) , 0.001
item: ('s8507257984367750231',)

item: ('u57782256373',) , 0.002
item: ('m1103189286',) , 0.002
item: ('m1269226635',) , 0.002
item: ('m988891860',) , 0.002
item: ('s14104859448908925354',) , 0.002
item: ('m1093661076',) , 0.002
item: ('u57782699329',) , 0.002
item: ('m1322434200',) , 0.002
item: ('s11921824146879545463',) , 0.002
item: ('m994462962',) , 0.002
item: ('m1283163060',) , 0.002
item: ('m1360527097',) , 0.002
item: ('m1360526365',) , 0.002
item: ('m1009051918',) , 0.002
item: ('m977218995',) , 0.002
item: ('m999706128',) , 0.002
item: ('m1322432613',) , 0.002
item: ('m1147188800',) , 0.002
item: ('s10472416650636786294',) , 0.002
item: ('m1090753072',) , 0.002
item: ('m1322433711',) , 0.002
item: ('m1322431923',) , 0.002
item: ('m1026166127',) , 0.002
item: ('m1308850794',) , 0.002
item: ('u84978987755',) , 0.002
item: ('m1322433315',) , 0.002
item: ('m1025555600',) , 0.002
item: ('m976781478',) , 0.002
item: ('m1044892082',) , 0.002
item: ('m1008144319',) , 0.002
item: ('vfcd47d7dbe0e4491b032c415d40a2b57'

item: ('m990773062',) , 0.003
item: ('m996961514',) , 0.003
item: ('m1322431572',) , 0.003
item: ('m1360527004',) , 0.003
item: ('u57801045108',) , 0.003
item: ('m1144128854',) , 0.003
item: ('m1011281491',) , 0.003
item: ('s10137660742710162855',) , 0.003
item: ('s10337275800949750662',) , 0.003
item: ('m1045336763',) , 0.003
item: ('m996469598',) , 0.003
item: ('m1019926213',) , 0.003
item: ('m1206846613',) , 0.003
item: ('u61133737624',) , 0.003
item: ('m1370492783',) , 0.003
item: ('m1021931069',) , 0.003
item: ('m1288194089',) , 0.003
item: ('u69109506449',) , 0.003
item: ('m1322433792',) , 0.003
item: ('m1146194582',) , 0.003
item: ('m1230406425',) , 0.003
item: ('m1322432025',) , 0.003
item: ('u85041194025',) , 0.003
item: ('m1322431632',) , 0.003
item: ('m1360527238',) , 0.003
item: ('m1322433906',) , 0.003
item: ('m1005962014',) , 0.003
item: ('m1307659018',) , 0.003
item: ('u56857652406',) , 0.003
item: ('m1034439308',) , 0.003
item: ('m1025511881',) , 0.003
item: ('m13049215

Rule: ('m1022054770',) ==> ('m1342110055',) , 0.308
Rule: ('m1167275033',) ==> ('m1023606622',) , 0.308
Rule: ('m1322432343',) ==> ('m1030190257',) , 0.308
Rule: ('m1322432343',) ==> ('m1322431596',) , 0.308
Rule: ('m988942596',) ==> ('m1032856732',) , 0.308
Rule: ('u62267748468',) ==> ('u58309976738',) , 0.308
Rule: ('m1370494604',) ==> ('m1360526302',) , 0.308
Rule: ('v78e7e42b371567d9f3c58214c5feeda1',) ==> ('s6907243864958674208',) , 0.308
Rule: ('m1322432343',) ==> ('m1034257022',) , 0.308
Rule: ('m973283628',) ==> ('m1304921531',) , 0.312
Rule: ('u57802663776',) ==> ('m1023606622',) , 0.312
Rule: ('u57474520359',) ==> ('m992285274',) , 0.312
Rule: ('u57802663776',) ==> ('s6907243864958674208',) , 0.312
Rule: ('u57474520359',) ==> ('m1023606622',) , 0.312
Rule: ('m1015907353',) ==> ('m998144258',) , 0.316
Rule: ('m1027438381',) ==> ('s15447946718183289597',) , 0.318
Rule: ('m1312164601',) ==> ('m1368905050',) , 0.333
Rule: ('m1017106267',) ==> ('m1030190257',) , 0.333
Rule: ('m113

In [119]:
dfs[dfs['sid'] == 'm1023606622'].head()[['resource', 'query']]

,resource,query
602,"{""data"": {""media"": {""artist"": [""赵雷""], ""title"":...",我想听帅一点的歌
717,"{""data"": {""media"": {""artist"": [""赵雷""], ""title"":...",成都
992,"{""data"": {""media"": {""artist"": [""赵雷""], ""title"":...",放一首成都
2195,"{""data"": {""media"": {""artist"": [""赵雷""], ""title"":...",播放成都
8843,"{""data"": {""media"": {""artist"": [""赵雷""], ""title"":...",播放音乐成都


In [196]:
df_rule = pd.read_csv('out_recomm_rules_20%.csv', header=None)

In [210]:
def get_sid_info(df, sid):
    return df.loc[sid].iloc[0]['content'] + df.loc[sid].iloc[0]['slots']

fd = open('out_rules_display.txt', 'w')
for index, row in df_rule.iterrows():
    for sid in row[1].split('|'):
        if sid == '':
            continue
        fd.write("<<< " + sid + ":" + get_sid_info(df_sidlookup, sid) + '\n')
    for sid in row[2].split('|'):
        if sid == '':
            continue
        fd.write(">>> " + sid + ":" + get_sid_info(df_sidlookup, sid) + '\n')
    fd.write('\n')
fd.close()